In [2]:
%matplotlib qt
import numpy as np 
import matplotlib.pyplot as plt
from spec_im_utils import gui_fname, plot_pl_summary, plot_si_bands
from spec_im import SpectralImage, PLSpectralImage, PL3DSpectralImage
from sklearn.decomposition import FastICA, PCA

In [3]:
fname = gui_fname()

In [4]:
si = PL3DSpectralImage(fname='D:/Chris/Google Drive/CoSES/Data/UV PL/190427_071515_oo_asi_hyperspec_3d_scan.h5')

Load from D:/Chris/Google Drive/CoSES/Data/UV PL/190427_071515_oo_asi_hyperspec_3d_scan.h5 complete.
8 x 31 x 31 spatial x 1044 spectral points


In [5]:
si = si[330:600]
si.plot()
plt.figure()
si.plot_spec()

ZeroDivisionError: float division by zero

In [ ]:
# Flatten full array for PCA fit
(nz, ny, nx, nf) = np.shape(si.spec_im)
si3d_array = np.zeros((nx*ny*nz, nf))
print(np.shape(si3d_array))
slice_list = si.get_slice_list()
for kk in range(len(slice_list)):
    homeslice = si.get_slice(slice_list[kk]).spec_im
    #print('homeslice', homeslice.shape)
    for ii in range(ny):
        for jj in range(nx):
            si3d_array[kk*nx*ny + ii*nx + jj,:] = homeslice[jj,ii,:]

In [ ]:
# Fit PCA to full array
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(si3d_array)

In [ ]:
# Plot explained variance ratio
plt.figure()
xvals = np.array(range(len(pca.explained_variance_ratio_)))+1
plt.semilogy(xvals, pca.explained_variance_ratio_, 'b.')
plt.xlabel('component')
plt.ylabel('explained variance ratio');

In [ ]:
# Transform full array using the PCA fit
pca_result = pca.transform(si3d_array)

In [ ]:
# Plot all PCA components
asdf = np.dot(pca_result, pca.components_)
plt.figure()
for kk in range(pca_result.shape[1]):
    plt.plot(si.spec_x_array, asdf[kk,:])

In [ ]:
# Fit FastICA for singular value decomposition on PCA loadings

fica = FastICA(n_components=20, max_iter=2000)
fica.fit(pca_result)

In [ ]:
# Transform the full PCA result
tf = fica.transform(pca_result)

In [ ]:
# Calculate mixed PCA components from ICA
new_comps = np.dot(fica.mixing_.T, pca.components_)

In [ ]:
# Plot each ICA component and its corresponding loading map
ii = 5
for kk in range(20):
#for kk in range(tf.shape[1]):
    plt.figure()
    plt.subplot(2,4,1)
    si._plot(np.reshape(tf[ii*nx*ny:(ii+1)*nx*ny, kk], (nx, ny)))
    plt.subplot(2,1,2)
    plt.plot(si.spec_x_array, asdf[kk,:])

In [ ]:
# Flatten one of the z slices for transformation with ICA
img = si.get_slice(slice_list[5]).spec_im
img_flat = np.zeros((nx*ny, nf))
for ii in range(ny):
    for jj in range(nx):
        img_flat[ii*nx+jj,:] = img[jj,ii,:]

In [ ]:
# Transform slice with PCA, transform PCA transform with ICA
print(img_flat.shape)
pca_result = pca.transform(img_flat)
print(pca_result.shape)
tf = fica.transform(pca_result)
print('transform shape', tf.shape)
print('mixing', fica.mixing_.shape)

In [ ]:
# Plot each ICA component and its corresponding loading map
for kk in range(20):
#for kk in range(tf.shape[1]):
    plt.figure()
    plt.subplot(2,4,1)
    si._plot(np.reshape(tf[:, kk], (nx, ny)))
    plt.subplot(2,1,2)
    plt.plot(si.spec_x_array, asdf[kk,:])

In [ ]:
# Plot laser of all slices to try to get a mask
ii = 0
for key in slice_list:
    ssi = si.get_slice(key)
    plot_si_bands(ssi, 
                  (340, 360, 'viridis', ''),
#                (2.5,2.6,'viridis',''), (2.6,2.7,'viridis',''), (2.7,2.8,'viridis',''), (2.8,2.9,'viridis',''), (2.9,3.0,'viridis',''),
#                (3.0,3.1,'viridis',''), (3.1,3.2,'viridis',''), (3.3,3.4,'viridis',''), (3.2,3.3,'viridis',''), (3.4,3.5,'viridis',''),
                no_of_cols=1, percentile=3)
    plt.gcf().tight_layout(h_pad=1)
    plt.suptitle('%d: z = %s mm' % (ii, key))
    ii += 1

In [ ]:
# Make the mask
plt.figure()
threshold = 5.45e4
mask = si.get_slice(slice_list[3])[340:360].spec_im.sum(axis=-1) < threshold
plt.imshow(mask)

In [ ]:
print(nx*ny, np.count_nonzero(mask.astype(int).flatten()))

In [ ]:
# Flatten full array for PCA fit
(nz, ny, nx, nf) = np.shape(si.spec_im)
mask_size = np.count_nonzero(mask)
masked_array = np.zeros((mask_size*nz, nf))
print(np.shape(masked_array))
slice_list = si.get_slice_list()
for kk in range(len(slice_list)):
    homeslice = si.get_slice(slice_list[kk]).spec_im
    masked_array[kk*mask_size:(kk+1)*mask_size,:] = np.reshape(homeslice[np.nonzero(mask)], (mask_size, nf))

In [ ]:
pca = PCA(n_components=0.99, svd_solver='full')
pca_result = pca.fit_transform(masked_array)
# Plot explained variance ratio
plt.figure()
xvals = np.array(range(len(pca.explained_variance_ratio_)))+1
plt.semilogy(xvals, pca.explained_variance_ratio_, 'b.')
plt.xlabel('component')
plt.ylabel('explained variance ratio');

In [ ]:
fica = FastICA(n_components=10, max_iter=2000)
tf = fica.fit_transform(pca_result)

In [ ]:
# Calculate mixed PCA components from ICA
new_comps = np.dot(fica.mixing_.T, pca.components_)

In [ ]:
ii = 2
print(tf.shape)
# Plot each ICA component and its corresponding loading map
#for kk in range(10):
#for ii in range(nz):
for ii in [5]:
    for kk in range(tf.shape[1]):
        comp = new_comps[kk, :]
        comp_fft = np.fft.fft(comp)
        comp_avg = np.average(comp)
        comp_fft_avg = np.average(comp_fft.real)
        freq = np.fft.fftfreq(comp.size, d=si.spec_x_array[1]-si.spec_x_array[0])
        fft_count = np.count_nonzero(np.abs(comp_fft[100:-100]) > 1000)
        label = '%d, avg = %0.2f, fft_average = %0.2f, %d' % (kk, comp_avg, comp_fft_avg, fft_count)
        print(label)
        #if np.abs(comp_fft_avg) < 10 and np.abs(comp_avg) > 8 and fft_count < 2:
        if True:
            plt.figure()
            plt.subplot(1,2,1)
            asdf = np.empty(nx*ny)
            asdf[:] = np.nan
            asdf[np.nonzero(mask.flatten())] = tf[ii*mask_size:(ii+1)*mask_size, kk] 
            asdf = np.reshape(asdf, (nx, ny))
            plt.title('component %d loading' % kk)
            si._plot(asdf)
            plt.subplot(2,2,2)
            plt.plot(si.spec_x_array, comp)
            plt.xlabel('$\lambda$ (nm)')
            plt.title('component %d' % kk)
            plt.subplot(2,2,4)
            plt.plot(freq, comp_fft.real,'b-')
            plt.plot(freq, comp_fft.imag,'r--')
            plt.xlabel('nm$^{-1}$')
            plt.title('component %d FFT' % kk)
            plt.suptitle('z = %s mm' % slice_list[ii])